## Import packages and define functions

In [261]:
import arcpy
import os
import tempfile
import glob
import datetime


In [262]:
# Define Function for spatial joins
def execute_spatial_joins(point_layer, input_wbd_layer, source, lake_id_field_name, search_radius_meters, output_point_layer_path):
    input_wbd_layer_name = str(source) + os.path.basename(input_wbd_layer).replace('.shp', '')
    search_radius_value = str(search_radius_meters) + " Meters"

    print(f"Processing: {input_wbd_layer_name}")

    # Define the path for the temporary layer for direct intersection
    temp_layer = os.path.join(temp_gdb, f'{input_wbd_layer_name}_temp_layer') ## EDK: Variable temp_gdb and also 'point_intersections' apparently undefined, But somehow the code still worked.
    # Define the path for the temporary layer for neighborhood spatial join
    dist_temp_layer = os.path.join(temp_gdb, f'{input_wbd_layer_name}_dist_temp_layer')
    # Define the path for the temporary layer for nearest features
    near_temp_layer = os.path.join(temp_gdb, f'{input_wbd_layer_name}_near_temp_layer')


    # Perform spatial join to identify points directly intersecting lakes
    arcpy.analysis.SpatialJoin(
        target_features=point_layer,
        join_features=input_wbd_layer,
        out_feature_class=temp_layer,
        join_operation="JOIN_ONE_TO_MANY",
        join_type="KEEP_COMMON",
        match_option="INTERSECT",
        search_radius=None,
        distance_field_name=""
    )
    

    # Record the total number of lakes that each point intersects with. Record the IDs of the lakes each point intersects with. 
    with arcpy.da.SearchCursor(temp_layer, ["TARGET_FID", "JOIN_FID", str(lake_id_field_name)]) as cursor:
        for row in cursor: # EDK: Might these row indexes change if you repeat this operation on a future version of this file that has more attributes or a different attribute order? Or maybe you have no choice and this is an arcpy quirk.
            point_fid = row[0]
            polygon_fid = row[1]
            lake_id = row[2]
            
            new_polygon_id = str(input_wbd_layer_name)+"_"+str(polygon_fid)+"_"+str(lake_id) 
            
            if point_fid not in point_intersections:
                point_intersections[point_fid] = {"count": 0, "ids": [], "dist_count": 0, "dist_ids": [], "near_id": [], "ndist_m": []}
            point_intersections[point_fid]["count"] += 1
            point_intersections[point_fid]["ids"].append(str(new_polygon_id))

            

    # Perform spatial join to identify lakes within 100 m of points
    arcpy.analysis.SpatialJoin( # EDK: Here is where arcGIS has a better alg than QGIS. in QGIS, Spatial join within a distance requires creating a buffered version of the joint features. This in turn, required projecting to an equal area coordinate system.
        target_features=point_layer,
        join_features=input_wbd_layer,
        out_feature_class=dist_temp_layer,
        join_operation="JOIN_ONE_TO_MANY",
        join_type="KEEP_COMMON",
        match_option="WITHIN_A_DISTANCE_GEODESIC",
        search_radius=search_radius_value,
        distance_field_name=""
    )

    # Record the total number of lakes that each point is within 100 m of. Record the IDs of the lakes within 100 m of each point. ## EDK can this snippet be generalized to a function?
    with arcpy.da.SearchCursor(dist_temp_layer, ["TARGET_FID", "JOIN_FID", str(lake_id_field_name)]) as cursor:
        for row in cursor:
            dist_point_fid = row[0]
            dist_polygon_fid = row[1]
            dist_lake_id = row[2]
            
            new_dist_polygon_id = str(input_wbd_layer_name)+"_"+str(dist_polygon_fid)+"_"+str(dist_lake_id) 
            
            if dist_point_fid not in point_intersections:
                point_intersections[dist_point_fid] = {"count": 0, "ids": [], "dist_count": 0, "dist_ids": [], "near_id": [], "ndist_m": []}
            point_intersections[dist_point_fid]["dist_count"] += 1
            point_intersections[dist_point_fid]["dist_ids"].append(str(new_dist_polygon_id)) # EDK: I wonder if this string of potentialy > 1 polygon IDs will be easier to parse, Or if you should use "tall" Data format where each point can appear in multiple rows, one for each polygon match. Just a thought – obviously the current method works for you.

    search_radius_value = str(search_radius_meters) + " Meters"

    # Generate Near Table to identify closest lakes within 100 m of points
    arcpy.analysis.GenerateNearTable(
        in_features=point_layer,
        near_features=input_wbd_layer,
        out_table=near_temp_layer,
        search_radius=search_radius_value,
        location="NO_LOCATION",
        angle="NO_ANGLE",
        closest="CLOSEST",
        closest_count=1, # EDK: I think this parameter is ignored if you have specified closest.
        method="GEODESIC",
        distance_unit="Meters"
    )
    
    # Create dictionary that lists lake IDs for each near_polygon_fid # EDK: Here is where I would recommend creating a common field name between Water body datasets so you can remove this if statement.
    if source == "PLD":
        identifier_field = "FID"
    else:
        identifier_field = "OBJECTID"
        
    near_fields = [str(identifier_field), str(lake_id_field_name)]
    near_features = {}
    with arcpy.da.SearchCursor(input_wbd_layer, near_fields) as cursor:
        for row in cursor:
            near_polygon_fid = row[0]
            near_lake_id = row[1]
            
            new_near_polygon_id = str(input_wbd_layer_name)+"_"+str(near_polygon_fid)+"_"+str(near_lake_id) 
            
            near_features[near_polygon_fid] = {"new_lake_id": new_near_polygon_id}
            
    # Record the nearest lake ID and distance (within 100 m) for each point
    with arcpy.da.SearchCursor(near_temp_layer, ["IN_FID", "NEAR_FID", "NEAR_DIST"]) as cursor:
        for row in cursor:
            near_point_fid = row[0]
            near_join_fid = row[1]
            near_distance = row[2]
            
            if near_point_fid not in point_intersections:
                point_intersections[near_point_fid] = {"count": 0, "ids": [], "dist_count": 0, "dist_ids": [], "near_id": [], "ndist_m": []} # EDK: This snippet Could also be generalized into a function, where you supply the names "near", "dist" and "" when you call it
            else:
                point_intersections[near_point_fid]["near_id"].append(str(near_features[near_join_fid]["new_lake_id"]))
                point_intersections[near_point_fid]["ndist_m"].append(str(near_distance))

    return point_intersections # EDK: I'd like to see this output

## Set base parameters

In [263]:
# Define basepath
basepath =  r"D:\__THOWARD\ABOVE_Project\Technical"
todays_date = datetime.date.today().strftime('%Y-%m-%d') # EDK: Good version control!
print(todays_date)


2024-07-28


In [265]:
# Path to point feature layer
point_layer_path = os.path.join(basepath, r"InputData\Vectors\LAKESHAPE\LAKESHAPE\effluxlakes.shp")

# Path to the output point feature layer
output_point_layer_path = os.path.join(basepath, r"OutputData\Vectors\PointFeatureSpatialJoins\effluxlakes_spatialjoins_{}.shp".format(todays_date))

# Copy point layer
arcpy.management.Copy(point_layer_path, output_point_layer_path)

# Create a spatial index for the point layer
arcpy.management.AddSpatialIndex(point_layer_path)

<Result 'D:\\__THOWARD\\ABOVE_Project\\Technical\\InputData\\Vectors\\LAKESHAPE\\LAKESHAPE\\effluxlakes.shp'>

## Conduct Spatial Joins with PLD Lakes

In [266]:
# Path to folder containing shapefiles of lake polygons
PLD_shapefile_folder = os.path.join(basepath, r"InputData\Vectors\PLD_TopoCat_dataset\PLD_TopoCat_dataset\PLD_lakes")

# List of all the shapefiles in the shapefile folder
PLD_shapefiles = glob.glob(os.path.join(PLD_shapefile_folder, '*.shp')) # EDK: If this loop over files is taking a long time or requires a lot of lines of code, perhaps you can merge these individual shaped files into a GDB (which doesn't have a feature limit like SHP)? The best tool for this is ogr2ogr on the command line, and should take up to ~20 minutes.

# Reduce number of shapefiles searched
###### ETHAN, CAN YOU PLEASE LET ME KNOW IF YOU THINK THAT I HAVE MISSED A SHAPEFILE?
endings = [
    '_71.shp',
    '_72.shp',
    '_74.shp',
    '_78.shp',    
    '_81.shp',    
    '_82.shp',    
    '_83.shp',    
    '_84.shp',    
    '_85.shp',
    '_86.shp', 
    '_91.shp'
]

PLD_shapefile_list = [shp for shp in PLD_shapefiles if any(shp.endswith(ending) for ending in endings)]

# Add new columns to the point layer for PLD 
arcpy.management.AddField(output_point_layer_path, 'PLDIntNum', 'LONG')
arcpy.management.AddField(output_point_layer_path, 'PLDIntIDs', 'TEXT', field_length=500)
arcpy.management.AddField(output_point_layer_path, 'PLD100mNum', 'LONG')
arcpy.management.AddField(output_point_layer_path, 'PLD100mIDs', 'TEXT', field_length=500)
arcpy.management.AddField(output_point_layer_path, 'PLDNdistID', 'TEXT', field_length=500)
arcpy.management.AddField(output_point_layer_path, 'PLDNdist_m', 'TEXT', field_length=500)

<Result 'D:\\__THOWARD\\ABOVE_Project\\Technical\\OutputData\\Vectors\\PointFeatureSpatialJoins\\effluxlakes_spatialjoins_2024-07-28.shp'>

In [267]:
# Clean up temporary files
arcpy.management.Delete("point_layer")
arcpy.management.Delete(temp_gdb)
# os.rmdir(temp_dir)

# Load the point feature layer
input_point_layer = arcpy.management.MakeFeatureLayer(point_layer_path, "point_layer")

# Create a temporary workspace
temp_dir = tempfile.mkdtemp()
temp_gdb = os.path.join(temp_dir, 'temp.gdb')
arcpy.management.CreateFileGDB(temp_dir, 'temp.gdb')


<Result 'C:\\Users\\thoward3\\AppData\\Local\\Temp\\ArcGISProTemp9828\\tmplyhequtm\\temp.gdb'>

In [268]:
## EDK in a notebook, can add cell magic %%time if you want to time this operation.
# Create a dictionary to store results for each point
point_intersections = {}

# Execute the function and update output point layer
## EDK: can use tqdm to give you a progress bar: for input_polygon_layer in tqdm.tqdm(PLD_shapefile_list): ...
for input_polygon_layer in PLD_shapefile_list:
    point_intersections = execute_spatial_joins(
                              point_layer = input_point_layer, 
                              input_wbd_layer = input_polygon_layer,
                              source = "PLD", 
                              lake_id_field_name = "lake_id", 
                              search_radius_meters = 100, 
                              output_point_layer_path = output_point_layer_path
                             )

    with arcpy.da.UpdateCursor(output_point_layer_path, ["FID", "PLDIntNum","PLDIntIDs", "PLD100mNum", "PLD100mIDs", "PLDNdistID", "PLDNdist_m"]) as cursor:
        for row in cursor:
            fid = row[0]
            if fid in point_intersections:
                row[1] = point_intersections[fid]["count"]
                row[2] = ','.join(point_intersections[fid]["ids"])
                row[3] = point_intersections[fid]["dist_count"]
                row[4] = ','.join(point_intersections[fid]["dist_ids"])
                row[5] = ','.join(point_intersections[fid]["near_id"])
                row[6] = ','.join(point_intersections[fid]["ndist_m"])

            else:
                row[1] = 0
                row[2] = ""
                row[3] = 0
                row[4] = ""
                row[5] = ""
                row[6] = ""
            cursor.updateRow(row)

    print("Processing complete. Output saved at: ", output_point_layer_path) ## EDK: If I understand correctly, this appends to the existing file. This is good, because it saves your work if the code crashes.
        
# Clean up temporary files
arcpy.management.Delete(temp_gdb)
os.rmdir(temp_dir)

Processing: PLDPLD_lakes_pfaf_71
Processing complete. Output saved at:  D:\__THOWARD\ABOVE_Project\Technical\OutputData\Vectors\PointFeatureSpatialJoins\effluxlakes_spatialjoins_2024-07-28.shp
Processing: PLDPLD_lakes_pfaf_72
Processing complete. Output saved at:  D:\__THOWARD\ABOVE_Project\Technical\OutputData\Vectors\PointFeatureSpatialJoins\effluxlakes_spatialjoins_2024-07-28.shp
Processing: PLDPLD_lakes_pfaf_74
Processing complete. Output saved at:  D:\__THOWARD\ABOVE_Project\Technical\OutputData\Vectors\PointFeatureSpatialJoins\effluxlakes_spatialjoins_2024-07-28.shp
Processing: PLDPLD_lakes_pfaf_78
Processing complete. Output saved at:  D:\__THOWARD\ABOVE_Project\Technical\OutputData\Vectors\PointFeatureSpatialJoins\effluxlakes_spatialjoins_2024-07-28.shp
Processing: PLDPLD_lakes_pfaf_81
Processing complete. Output saved at:  D:\__THOWARD\ABOVE_Project\Technical\OutputData\Vectors\PointFeatureSpatialJoins\effluxlakes_spatialjoins_2024-07-28.shp
Processing: PLDPLD_lakes_pfaf_82
Pr

## Conduct Spatial Joins with SUI Lakes

In [269]:
# Path to the SUI lake polygon feature layer
sui_WBD_layer = os.path.join(basepath, r"InputData\Databases\water_edited.gdb\WBD")
sui_layer_list = [sui_WBD_layer]

# Add new columns to the point layer for SUI
arcpy.management.AddField(output_point_layer_path, 'SuiIntNum', 'LONG')
arcpy.management.AddField(output_point_layer_path, 'SuiIntIDs', 'TEXT', field_length=500)
arcpy.management.AddField(output_point_layer_path, 'Sui100mNum', 'LONG')
arcpy.management.AddField(output_point_layer_path, 'Sui100mIDs', 'TEXT', field_length=500)
arcpy.management.AddField(output_point_layer_path, 'SuiNdistID', 'TEXT', field_length=500)
arcpy.management.AddField(output_point_layer_path, 'SuiNdist_m', 'TEXT', field_length=500)

<Result 'D:\\__THOWARD\\ABOVE_Project\\Technical\\OutputData\\Vectors\\PointFeatureSpatialJoins\\effluxlakes_spatialjoins_2024-07-28.shp'>

In [270]:
# Clean up temporary files
arcpy.management.Delete("point_layer")

# Load the point feature layer
input_point_layer = arcpy.management.MakeFeatureLayer(point_layer_path, "point_layer")

# Create a temporary workspace
temp_dir = tempfile.mkdtemp()
temp_gdb = os.path.join(temp_dir, 'temp.gdb')
arcpy.management.CreateFileGDB(temp_dir, 'temp.gdb')


<Result 'C:\\Users\\thoward3\\AppData\\Local\\Temp\\ArcGISProTemp9828\\tmp4uw35i5i\\temp.gdb'>

In [271]:
# Create a dictionary to store results for each point
point_intersections = {}

# Execute the function and update output point layer
for input_polygon_layer in sui_layer_list:
    point_intersections = execute_spatial_joins(
                              point_layer = input_point_layer, 
                              input_wbd_layer = input_polygon_layer,
                              source = "Sui", 
                              lake_id_field_name = "SUI_ID", 
                              search_radius_meters = 100, 
                              output_point_layer_path = output_point_layer_path
                             )

    # EDK: This text can also be generalized to a function to avoid repeating similar text blocks. YOu can generate field names using f-strings: wdb_name='Sui'; f"{wdb_name}IntNum", f"{wdb_name}IntIDs",] etc.
    with arcpy.da.UpdateCursor(output_point_layer_path, ["FID", "SuiIntNum", "SuiIntIDs", "Sui100mNum", "Sui100mIDs", "SuiNdistID", "SuiNdist_m"]) as cursor:
        for row in cursor:
            fid = row[0]
            if fid in point_intersections:
                row[1] = point_intersections[fid]["count"]
                row[2] = ','.join(point_intersections[fid]["ids"])
                row[3] = point_intersections[fid]["dist_count"]
                row[4] = ','.join(point_intersections[fid]["dist_ids"])
                row[5] = ','.join(point_intersections[fid]["near_id"])
                row[6] = ','.join(point_intersections[fid]["ndist_m"])

            else:
                row[1] = 0
                row[2] = ""
                row[3] = 0
                row[4] = ""
                row[5] = ""
                row[6] = ""
            cursor.updateRow(row)

    print("Processing complete. Output saved at: ", output_point_layer_path)
        
# Clean up temporary files
arcpy.management.Delete(temp_gdb)
os.rmdir(temp_dir)

Processing: SuiWBD
Processing complete. Output saved at:  D:\__THOWARD\ABOVE_Project\Technical\OutputData\Vectors\PointFeatureSpatialJoins\effluxlakes_spatialjoins_2024-07-28.shp


In [272]:
# Add the new field to the shapefile
new_field_name = "IntDescrpt"
arcpy.management.AddField(output_point_layer_path, new_field_name, "TEXT", field_length=500)

# Update the new field based on the conditions
with arcpy.da.UpdateCursor(output_point_layer_path, ["PLDIntNum", "SuiIntNum", "PLD100mNum", "Sui100mNum", new_field_name]) as cursor:
    for row in cursor:
        PLDIntNum = row[0]
        SuiIntNum = row[1]
        PLD100mNum = row[2]
        Sui100mNum = row[3]
        
        if PLDIntNum > 1 and SuiIntNum > 1:
            row[4] = "Intersects multiple lakes in both PLD and Sui" # EDK: Love that you made this proof by writing out the results in plain English! It's easy for me to get confused when working with tabular data.
        elif PLDIntNum > 1 and SuiIntNum < 1:
            row[4] = "Intersects multiple lakes in PLD but not in Sui"
        elif PLDIntNum < 1 and SuiIntNum > 1:
            row[4] = "Intersects multiple lakes in Sui but not in PLD"
        elif PLDIntNum == 1 and SuiIntNum == 1:
            row[4] = "Intersects lake in PLD and Sui"
        elif PLDIntNum == 1 and SuiIntNum == 0:
            row[4] = "Intersects lake in PLD but not in Sui"
        elif PLDIntNum == 0 and SuiIntNum == 1:
            row[4] = "Intersects lake in Sui but not in PLD"
        elif PLDIntNum == 0 and SuiIntNum == 0 and PLD100mNum > 0 and Sui100mNum > 0:
            row[4] = "Within 100 m of lakes in PLD and in Sui"
        elif PLDIntNum == 0 and SuiIntNum == 0 and PLD100mNum > 0 and Sui100mNum == 0:
            row[4] = "Within 100 m of lakes in PLD but not in Sui"
        elif PLDIntNum == 0 and SuiIntNum == 0 and PLD100mNum == 0 and Sui100mNum > 0:
            row[4] = "Within 100 m of lakes in Sui but not in PLD"
        elif PLDIntNum == 0 and SuiIntNum == 0 and PLD100mNum == 0 and Sui100mNum == 0:
            row[4] = "Not within 100 m of previously identified waterbody"
        cursor.updateRow(row)

print("New field added and updated successfully.")


New field added and updated successfully.
